In [ ]:
# Install BigQuery client library (run this cell once)
%pip install google-cloud-bigquery --quiet
%pip install bigquery-magics --quiet
%pip install db-dtypes --quiet
# Import libraries
from google.cloud import bigquery
import pandas as pd
%load_ext bigquery_magics
import db_dtypes
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "salesforce-465614-2cf9e37da64b.json"
from google.cloud import bigquery
# Initialize BigQuery client
client = bigquery.Client()


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
The bigquery_magics extension is already loaded. To reload it, use:
  %reload_ext bigquery_magics



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


<b>Truncate Stage tables before loading from CSV's</b>

In [ ]:
%%bqsql
truncate table salesforce-465614.cust_analytics.stage_transactions;
truncate table salesforce-465614.cust_analytics.stage_customers;
truncate table salesforce-465614.cust_analytics.stage_products;
truncate table salesforce-465614.cust_analytics.stage_stores;

Query is running:   0%|          |

""


<b>Entry in ETL Job Monitoring table</b>

In [4]:
from datetime import datetime
from google.cloud import bigquery

# Set monitoring table and module name
monitoring_table = "salesforce-465614.cust_analytics.etl_job_monitoring"
module_name = "File to Stage Layer"
job_run_date = datetime.utcnow().date()
start_time = datetime.utcnow()
# Insert start record
insert_start = f"""
INSERT INTO `{monitoring_table}` (job_run_date, module_name, start_time, status)
VALUES (DATE('{job_run_date}'), '{module_name}', TIMESTAMP('{start_time}'), 'Running')
"""
client.query(insert_start).result()

C:\Users\Home\AppData\Local\Temp\ipykernel_20164\1991522017.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  job_run_date = datetime.utcnow().date()
C:\Users\Home\AppData\Local\Temp\ipykernel_20164\1991522017.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_time = datetime.utcnow()


<b>Loading Stage Tables</b>

In [ ]:
from google.cloud import bigquery
# Initialize BigQuery client
client = bigquery.Client()
# Define dataset and table names
dataset_id = "cust_analytics"  
file_path = "v2/"
# Mapping of file names to table names
# "DimCustomer.csv": "stage_customers",
# "DimProduct.csv": "stage_products",
# "DimStore.csv": "stage_stores"
file_table_map = {
    "Transactions.csv": "stage_transactions",
    "Customer.csv": "stage_customers",
    "Product.csv": "stage_products",
    "Store.csv": "stage_stores"
}

# Load each CSV into its respective BigQuery table
for file_name, table_name in file_table_map.items():
    table_id = f"{client.project}.{dataset_id}.{table_name}"
    full_file_path = os.path.join(file_path, file_name)
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        autodetect=True,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
    )
    with open(full_file_path, "rb") as source_file:
        load_job = client.load_table_from_file(source_file, table_id, job_config=job_config)
    load_job.result()  # Wait for the job to complete
    print(f"Loaded {file_name} to {table_id}")
try:
    end_time = datetime.utcnow()
    update_success = f"""
    UPDATE `{monitoring_table}`
    SET end_time = TIMESTAMP('{end_time}'), status = 'Success'
    WHERE job_run_date = DATE('{job_run_date}') AND module_name = '{module_name}' AND start_time = TIMESTAMP('{start_time}')
    """
    client.query(update_success).result()
except Exception as e:
    # Update status as Failed
    fail_time = datetime.utcnow()
    update_failed = f"""
    UPDATE `{monitoring_table}`
    SET end_time = TIMESTAMP('{fail_time}'), status = 'Failed'
    WHERE job_run_date = DATE('{job_run_date}') AND module_name = '{module_name}' AND start_time = TIMESTAMP('{start_time}')
    """
    client.query(update_failed).result()
    print("ETL job failed:", e)
    raise

Loaded Transactions.csv to salesforce-465614.cust_analytics.stage_transactions


C:\Users\Home\AppData\Local\Temp\ipykernel_20164\3381528795.py:31: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_time = datetime.utcnow()


<b>Data Cleaning before loading to Persistant Tables</b>

In [ ]:
# List of stage tables to check
stage_tables = [
    "stage_customers",
    "stage_products",
    "stage_transactions",
    "stage_stores"
]
# Dictionary to store not-null counts for each table
not_null_counts_dict = {}
for table in stage_tables:
    query = f"SELECT * FROM `{client.project}.{dataset_id}.{table}`"
    df = client.query(query).to_dataframe()
    not_null_counts_dict[table] = df.notnull().sum()
# Display not-null counts for each table
for table, counts in not_null_counts_dict.items():
    print(f"Not-null counts for {table}:")
    print(counts)
    print("-" * 40)

c:\Users\Home\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Not-null counts for stage_customers:
customer_id        2000
first_name         2000
last_name          2000
gender             2000
age                2000
signup_date        2000
loyalty_program    2000
email              2000
city               2000
state              2000
country            2000
dtype: int64
----------------------------------------
Not-null counts for stage_products:
product_id      200
product_name    200
category        200
sub_category    200
price           200
dtype: int64
----------------------------------------
Not-null counts for stage_transactions:
Transaction_id    20000
Customer_ID       20000
Product_id        20000
Store_Location    20000
Purchase_Date     20000
Product_Type      20000
Unit_Price        20000
Quantity          20000
Total_Price       20000
Payment_Method    20000
Channel           20000
Order_Status      20000
Loyalty_Member    20000
Age               20000
Gender            20000
City              20000
dtype: int64
------------------

In [ ]:
# Script to delete duplicates in each stage table based on all columns
stage_tables = [
    "stage_customers",
    "stage_products",
    "stage_transactions",
    "stage_stores"
]

# 1. Check for primary key uniqueness
for table in stage_tables:
    table_ref = f"{client.project}.{dataset_id}.{table}"
    schema = client.get_table(table_ref).schema
    pk_column = schema[0].name  # Assuming first column is primary key
    pk_check_query = f"""
    SELECT {pk_column}, COUNT(*) as count
    FROM `{table_ref}`
    GROUP BY {pk_column}
    HAVING count > 1
    """
    pk_duplicates = client.query(pk_check_query).to_dataframe()
    print(f"Primary key duplicates in {table}:", len(pk_duplicates))


Primary key duplicates in stage_customers: 0
Primary key duplicates in stage_products: 0
Primary key duplicates in stage_transactions: 0
Primary key duplicates in stage_stores: 0
Orphaned transaction records: 0
Null values in stage_customers.customer_id: 0
Null values in stage_customers.first_name: 0
Null values in stage_customers.last_name: 0
Null values in stage_customers.gender: 0
Null values in stage_customers.age: 0
Null values in stage_customers.signup_date: 0
Null values in stage_customers.loyalty_program: 0
Null values in stage_customers.email: 0
Null values in stage_customers.city: 0
Null values in stage_customers.state: 0
Null values in stage_customers.country: 0
Null values in stage_products.product_id: 0
Null values in stage_products.product_name: 0
Null values in stage_products.category: 0
Null values in stage_products.sub_category: 0
Null values in stage_products.price: 0
Null values in stage_transactions.Transaction_id: 0
Null values in stage_transactions.Customer_ID: 0


BadRequest: 400 Partitioning by expressions of type FLOAT64 is not allowed at [4:101]; reason: invalidQuery, location: query, message: Partitioning by expressions of type FLOAT64 is not allowed at [4:101]

Location: US
Job ID: 750846a6-eaba-4104-82a5-6c016f045012


In [3]:
%%bqsql
select distinct payment_method from salesforce-465614.cust_analytics.stage_transactions;

Query is running:   0%|          |

Downloading:   0%|          |

,payment_method
0,Credit Card
1,Paypal
2,Cash
3,PayPal
4,Bank Transfer
5,Debit Card


In [4]:
%%bqsql
update salesforce-465614.cust_analytics.stage_transactions
set payment_method = 'PayPal'
where lower(payment_method) = 'paypal'

Query is running:   0%|          |

""
